In [1]:
import os
import pandas as pd
import numpy as np

os.chdir('../spatialdm/')
# from main import *
from plottings import *

In [2]:
from main import *


In [39]:
from main_sparse import *


In [3]:
import anndata as ann
import scanpy as sc
import matplotlib.pyplot as plt
import time

In [22]:
from scipy import spatial

In [4]:
adata = sc.read_h5ad('/home/yoyo/1_spatial/2_BGI_mouse/E16.5_E1S3.MOSTA.h5ad')

In [10]:
adata

AnnData object with n_obs × n_vars = 10000 × 28579
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'annotation', 'Regulon - 2610044O15Rik8', 'Regulon - 5730507C01Rik', 'Regulon - A1cf', 'Regulon - AW146154', 'Regulon - Ahctf1', 'Regulon - Alx1', 'Regulon - Alx3', 'Regulon - Alx4', 'Regulon - Apex1', 'Regulon - Arid3a', 'Regulon - Arnt', 'Regulon - Arnt2', 'Regulon - Arx', 'Regulon - Atf1', 'Regulon - Atf2', 'Regulon - Atf4', 'Regulon - Atf5', 'Regulon - Atf6', 'Regulon - Banp', 'Regulon - Barhl1', 'Regulon - Barhl2', 'Regulon - Barx1', 'Regulon - Barx2', 'Regulon - Batf', 'Regulon - Bbx', 'Regulon - Bcl11a', 'Regulon - Bclaf1', 'Regulon - Bhlha15', 'Regulon - Bhlhe22', 'Regulon - Bhlhe40', 'Regulon - Bmyc', 'Regulon - Borcs8', 'Regulon - Brca1', 'Regulon - Brf1', 'Regulon - Bsx', 'Regulon - Carf', 'Regulon - Cbfb', 'Regulon - Cdc5l', 'Regulon - Cdx1', 'Regulon - Cdx2', 'Regulon - Cebpa', 'Regulon - Cebpb', 'Regulon - Cebpd', 'Regulon - Ceb

In [5]:
spatialcoods = pd.DataFrame(adata.obsm['spatial'])
spatialcoods.columns = ['x', 'y']

exp = pd.DataFrame(adata.X.todense(), columns=adata.var_names, index=spatialcoods.index)

In [ ]:
# spa = adata.obsm['spatial']
# spa=pd.DataFrame(spa, columns=['x','y'])

# spa = spa.transpose()

# spa.loc[:,np.arange(50000,1000000,1)]=np.random.randint(-3000,1000,(2,950000))

# spa = spa.transpose()

In [7]:
lens = [10000, 7500, 5000, 2500, 1000]
i=3
exp=exp.loc[:lens[i],:]
spatialcoods=spatialcoods.loc[:lens[i],:]
sample = SpatialDM(exp, spatialcoods)  

In [ ]:
times1 = []
times2 = []
sizes = []

lens = [10000, 7500, 5000, 2500, 1000]
for i in range(5):
    exp=exp.loc[:lens[i],:]
    spatialcoods=spatialcoods.loc[:lens[i],:]
    sample = SpatialDM(exp, spatialcoods)  
    sample.weight_matrix(l=75, cutoff=0.2, single_cell=False) # weight_matrix by rbf kernel
    print('cp_w')    
    sample.extract_lr('mouse', '/home/yoyo/1_spatial', min_cell=10)      # find overlapping LRs from CellChatDB
    print('cp_lr')    
    t0 = time.time()
    sample.spatialdm_global(1000, select_num=np.arange(50), method='permutation',nproc=50)     # global Moran selection
    t1 = time.time()
    times1.append(t1-t0)
    sample.spatialdm_global(1000, select_num=np.arange(50), method='z-score',nproc=50)     # global Moran selection
    t2 = time.time()
    times2.append(t2-t1)
    sizes.append(sample.no_pairs)
    print('permutation: ')
    print(times1)
    print('z-score: ')
    print(times2)

checkpoint = dict(times1=times1, times2=times2, sizes=sizes)

In [20]:
times1

[252.92526602745056, 235.67109322547913]

In [21]:
times2

[4.314467668533325, 4.03339695930481]

In [ ]:
for k, v in checkpoint.items():
    np.save('{}_re.npy'.format(k), np.array(v, dtype=object))
    print('Successfully save perm_dir/{} ...'.format(k))

## sparse

In [14]:
# datadir = '/home/yoyo/1_spatial/1_Tumor/data/'
# exp=pd.read_csv(datadir + 'logcounts.csv',header=0,index_col=0)
# exp = exp.transpose()
# spatialcoord = pd.read_csv(datadir+'spatialcoord.csv', index_col=0)
# spatialcoord.columns = ['x', 'y']
# exp.index = spatialcoord.index


In [122]:
# res=sample.global_res.merge(sps.global_res, left_index=True, right_index=True)

In [124]:
# res.columns=['ligand_x', 'receptor_x', 'z_pval', 'perm_pval', 'ligand_y',
#        'receptor_y', 'z_pval_sparse', 'perm_pval_sparse']

In [132]:
# from main_sparse2 import *

In [133]:
# sps = SpatialDM(sample.exp, sample.spatialcoord)  
# sps.rbf_d = csc_matrix(pdist_res)
# print('cp_w')    
# sps.extract_lr('human', '/home/yoyo/1_spatial', min_cell=10)      # find overlapping LRs from CellChatDB
# print('cp_lr')    
# t0 = time.time()
# sps.spatialdm_global(100, select_num=np.arange(100), method='both',nproc=50)     # global Moran selection
# t1 = time.time()

cp_w
cp_lr


/home/yoyo/1_spatial/1_Tumor/git_rep/SpatialDM/spatialdm/main_sparse2.py:133: RuntimeWarning: invalid value encountered in double_scalars
  self.st = self.v ** (1 / 2)


sub
0
0pairs global selection finished in: 0.024596452713012695
sub
1
1pairs global selection finished in: 0.01836371421813965
sub
2
2pairs global selection finished in: 0.020919322967529297
sub
3
3pairs global selection finished in: 0.02090001106262207
sub
4
4pairs global selection finished in: 0.018825769424438477
sub
5
5pairs global selection finished in: 0.01797342300415039
sub
6
6pairs global selection finished in: 0.018819332122802734
sub
7
7pairs global selection finished in: 0.017947673797607422
sub
8
8pairs global selection finished in: 0.019181013107299805
sub
9
9pairs global selection finished in: 0.01812744140625
sub
10
10pairs global selection finished in: 0.02234625816345215
sub
11
11pairs global selection finished in: 0.018474817276000977
sub
12
12pairs global selection finished in: 0.018585205078125
sub
13
13pairs global selection finished in: 0.017711639404296875
sub
14
14pairs global selection finished in: 0.019115209579467773
sub
15
15pairs global selection finished 